# Cloud vertical classes 

Function that classifies the vertical cloud cover structure based on high-mid-low cloud cover and return a mask with the cloud classes.

input: 
    -Model output of high-, mid- and low-level cloud cover
    -Provide a cloud cover threshold to clasify a layer as cloudy or not
    
output:
    -1-D matrix of the cloud class for each column
    
The 1-D matrix will consist of 8 classes as follows:
        CL1: H          CL2: M          CL3: L          CL4: HM         CL5: ML         CL6: HL        CL7: HML        CL8: clear-sky       

In [1]:
import xarray as xr
import numpy as np
from cartopy import crs as ccrs
from matplotlib import pyplot as plt
import psutil

from tempfile import NamedTemporaryFile, TemporaryDirectory # Creating temporary Files/Dirs
import dask # Distributed data libary
from dask_jobqueue import SLURMCluster # Setting up distributed memories via slurm
from distributed import Client, progress, wait # Libaray to orchestrate distributed resources

In [2]:
# Set some user specific variables
account_name = 'bb1018'
partition = 'compute'
job_name = 'nawdexProc_george' # Job name that is submitted via sbatch
memory = '64GiB' # Max memory per node that is going to be used - this depends on the partition
cores = 48 # Max number of cores per that are reserved - also partition dependent
walltime = '01:00:00' #'12:00:00' # Walltime - also partition dependent

In [7]:
scratch_dir = '/scratch/b/b380796/' # Define the users scratch dir
# Create a temp directory where the output of distributed cluster will be written to, after this notebook
# is closed the temp directory will be closed
dask_scratch_dir = TemporaryDirectory(dir=scratch_dir, prefix=job_name)
cluster = SLURMCluster(memory=memory,
                       cores=cores,
                       project=account_name,
                       walltime=walltime,
                       queue=partition,
                       name=job_name,
                       processes=8,
                       scheduler_options={'dashboard_address': ':12435'},
                       local_directory=dask_scratch_dir.name,
                       job_extra=[f'-J {job_name}', 
                                  f'-D {dask_scratch_dir.name}',
                                  f'--begin=now',
                                  f'--output={dask_scratch_dir.name}/LOG_cluster.%j.o',
                                  f'--output={dask_scratch_dir.name}/LOG_cluster.%j.o'
                                 ],
                       interface='ib0')

/pf/b/b380459/conda-envs/Nawdex-Hackathon/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33133 instead
  warnings.warn(


In [4]:
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -A bb1018
#SBATCH -n 1
#SBATCH --cpus-per-task=48
#SBATCH --mem=64G
#SBATCH -t 01:00:00
#SBATCH -J nawdexProc_george
#SBATCH -D /scratch/b/b380796/nawdexProc_georgeamhh_9ra
#SBATCH --begin=now
#SBATCH --output=/scratch/b/b380796/nawdexProc_georgeamhh_9ra/LOG_cluster.%j.o
#SBATCH --output=/scratch/b/b380796/nawdexProc_georgeamhh_9ra/LOG_cluster.%j.o

JOB_ID=${SLURM_JOB_ID%;*}

/pf/b/b380459/conda-envs/Nawdex-Hackathon/bin/python3 -m distributed.cli.dask_worker tcp://10.50.39.196:45585 --nthreads 6 --nprocs 8 --memory-limit 8.59GB --name name --nanny --death-timeout 60 --local-directory /scratch/b/b380796/nawdexProc_georgeamhh_9ra --interface ib0



In [5]:
cluster.scale(jobs=1)
cluster

In [6]:
! squeue -u $USER --start

             JOBID PARTITION     NAME     USER ST          START_TIME  NODES SCHEDNODES           NODELIST(REASON)


In [13]:
dask_client = Client(cluster)
dask_client

Client Scheduler: tcp://10.50.39.93:35144 Dashboard: http://10.50.39.93:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [14]:
def load_iconsimulation(expid,gridres):
    
    print('Working on loading data for', expid)
    path  = '/scratch/b/b380459/icon_4_hackathon/'
    
    # 2d datasets
    fname = path+'/'+expid+'/'+expid+'_2016*_2d_30min_DOM01_ML_*.nc'
    print(fname)
    ds_2d_30min = ( xr.open_mfdataset(fname,
                                      combine='by_coords',parallel=True, 
                                      engine='h5netcdf', chunks={'time': 1})
                    [['clct', 'tqv_dia', 'tqc_dia', 'tqi_dia']] )
    
    # grid dataset
    fname = path+'/grids/icon-grid_nawdex_78w40e23n80n_'+gridres+'.nc'
    ds_grid = ( xr.open_dataset(fname)
               [['cell_area','clat','clon','clon_vertices','clat_vertices',
                 'vlon','vlat','vertex_of_cell', 'vertex']].rename({'cell': 'ncells'}) ) 
    # we need to subtract -1 from vertex_of_cell as python starts counting at 0, but fortran starts at 1
    ds_grid['vertex_of_cell'] = ds_grid['vertex_of_cell'] - 1 
   
    # get land-sea mask from extpar data
    fname = path+'/extpar/extpar_icon-grid_nawdex_78w40e23n80n_'+gridres+'_bitmap.nc'
    ds_extp = xr.open_dataset(fname)[['FR_LAND']].rename({'cell': 'ncells'})
    
    # merge datasets
    ds = xr.merge([ds_2d_30min, ds_grid, ds_extp])
        
    # convert grid from radians to degrees
    ds['clon'] = np.rad2deg(ds['clon'])
    ds['clat'] = np.rad2deg(ds['clat'])
    ds['vlon'] = np.rad2deg(ds['vlon'])
    ds['vlat'] = np.rad2deg(ds['vlat'])
    ds['clon_vertices'] = np.rad2deg(ds['clon_vertices'])
    ds['clat_vertices'] = np.rad2deg(ds['clat_vertices'])
    
    return ds

In [15]:
# function that prints out the memory used
def print_memory(msg=None):
    process = psutil.Process()
    if (msg):
        print(msg, ':', 'memory =', np.round(process.memory_info().rss/(1024*1024)), 'MB')
    else:
        print('memory =', np.round(process.memory_info().rss/(1024*1024)), 'MB')
print_memory()

memory = 856.0 MB


In [16]:
ds = load_iconsimulation('nawdexnwp-80km-mis-0001','R80000m')

Working on loading data for nawdexnwp-80km-mis-0001
/scratch/b/b380459/icon_4_hackathon//nawdexnwp-80km-mis-0001/nawdexnwp-80km-mis-0001_2016*_2d_30min_DOM01_ML_*.nc


We plot the data 

In [ ]:
plt.tricontourf(ds.clon, ds.clat, ds['clct'].isel(time=80));
plt.colorbar();

In [5]:
# we load all the timesteps for the 80km resolution
ipath='/scratch/b/b380459/icon_4_hackathon/nawdexnwp-80km-mis-0001/'
ifile = 'nawdexnwp-80km-mis-0001_2016092200_2d_30min_*.nc'

#ipath='/scratch/b/b380459/icon_4_hackathon/nawdexnwp-40km-mis-0001/'
#ifile = 'nawdexnwp-40km-mis-0001_2016092200_2d_30min_*.nc'

#ipath='/scratch/b/b380459/icon_4_hackathon/nawdexnwp-20km-mis-0001/'
#ifile = 'nawdexnwp-20km-mis-0001_2016092200_2d_30min_*.nc'

#ipath='/scratch/b/b380459/icon_4_hackathon/nawdexnwp-10km-mis-0001/'
#ifile = 'nawdexnwp-10km-mis-0001_2016092200_2d_30min_*.nc'

#ipath='/scratch/b/b380459/icon_4_hackathon/nawdexnwp-5km-mis-0001/'
#ifile = 'nawdexnwp-5km-mis-0001_2016092200_2d_30min_*.nc'

#ipath='/scratch/b/b380459/icon_4_hackathon/nawdexnwp-2km-mis-0001/'
#ifile = 'nawdexnwp-2km-mis-0001_2016092200_2d_30min_*.nc'

data = xr.open_mfdataset(ipath+ifile)
data
print_memory()

memory = 855.0 MB


In [ ]:
#test_data = data.sel(time='2016-09-22T18:00:00.000000000')
#test_data

In [ ]:
# mask the land areas and the domain boundaries using the openoceanmask 

def load_openoceanmask(expid):
    path  = '/scratch/b/b380459/icon_4_hackathon/'
    fname = path+'/openocean_masks/'+expid+'_openoceanmask.nc'
    return xr.open_dataset(fname)['mask_openocean']

da_oom  = load_openoceanmask('nawdexnwp-80km-mis-0001')
#da_oom  = load_openoceanmask('nawdexnwp-40km-mis-0001')
#da_oom  = load_openoceanmask('nawdexnwp-20km-mis-0001')
#da_oom  = load_openoceanmask('nawdexnwp-10km-mis-0001')
#da_oom  = load_openoceanmask('nawdexnwp-5km-mis-0001')
#da_oom  = load_openoceanmask('nawdexnwp-2km-mis-0001')


index = np.where(da_oom==1)[0]
data = data.isel(ncells=index)

In [ ]:
# we set a threshold for the cloud cover to identify cloudy and clear sky grid boxes
thres=10

# function that gets 2-D cloud cover data of High, Mid and Low level cloud cover and creates classes for the vertical column
def cloud_class(cc_data,thres):
    # CL1: H 
    # CL2: M 
    # CL3: L 
    # CL4: HM 
    # CL5: ML 
    # CL6: HL 
    # CL7: HML 
    # CL8: clear-sky
    
    # Create the vertical cloud classes based on the threshold (thres)
    cl1     = cc_data.where((cc_data.clch>thres) & (cc_data.clcm<thres) & (cc_data.clcl<thres)) # H
    cl1_num = cl1.where(xr.ufuncs.isnan(cl1.clch),other=1)
    cl1_num = cl1_num.where((xr.ufuncs.isnan(cl1_num.clch))==False,other=0)

    cl2 = cc_data.where((cc_data.clch<thres) & (cc_data.clcm>thres) & (cc_data.clcl<thres)) # M
    cl2_num = cl2.where(xr.ufuncs.isnan(cl2.clch),other=2)
    cl2_num = cl2_num.where((xr.ufuncs.isnan(cl2_num.clch))==False,other=0)

    
    cl3 = cc_data.where((cc_data.clch<thres) & (cc_data.clcm<thres) & (cc_data.clcl>thres)) # L
    cl3_num = cl3.where(xr.ufuncs.isnan(cl3.clch),other=3)
    cl3_num = cl3_num.where((xr.ufuncs.isnan(cl3_num.clch))==False,other=0)

    
    cl4 = cc_data.where((cc_data.clch>thres) & (cc_data.clcm>thres) & (cc_data.clcl<thres)) # HM
    cl4_num = cl4.where(xr.ufuncs.isnan(cl4.clch),other=4)
    cl4_num = cl4_num.where((xr.ufuncs.isnan(cl4_num.clch))==False,other=0)

    
    cl5 = cc_data.where((cc_data.clch<thres) & (cc_data.clcm>thres) & (cc_data.clcl>thres)) # ML
    cl5_num = cl5.where(xr.ufuncs.isnan(cl5.clch),other=5)
    cl5_num = cl5_num.where((xr.ufuncs.isnan(cl5_num.clch))==False,other=0)


    cl6 = cc_data.where((cc_data.clch>thres) & (cc_data.clcm<thres) & (cc_data.clcl>thres)) # HL
    cl6_num = cl6.where(xr.ufuncs.isnan(cl6.clch),other=6)
    cl6_num = cl6_num.where((xr.ufuncs.isnan(cl6_num.clch))==False,other=0)

    
    cl7 = cc_data.where((cc_data.clch>thres) & (cc_data.clcm>thres) & (cc_data.clcl>thres)) # HML
    cl7_num = cl7.where(xr.ufuncs.isnan(cl7.clch),other=7)
    cl7_num = cl7_num.where((xr.ufuncs.isnan(cl7_num.clch))==False,other=0)

    
    cl8 = cc_data.where((cc_data.clch<thres) & (cc_data.clcm<thres) & (cc_data.clcl<thres)) # clear-sky
    cl8_num = cl8.where(xr.ufuncs.isnan(cl8.clch),other=8)
    cl8_num = cl8_num.where((xr.ufuncs.isnan(cl8_num.clch))==False,other=0)

    # we sum all the classes to one array to make a mask for the entire domain based on the cloud classification
    cloud_class_mask   = cl1+cl2+cl3+cl4+cl5+cl6+cl7+cl8 
    cloud_class_number = cl1_num+cl2_num+cl3_num+cl4_num+cl5_num+cl6_num+cl7_num+cl8_num
    return cloud_class_mask, cloud_class_number

cloud_class_array, cloud_class_number = cloud_class(data,thres)

print('###################################################################')
print('TEST OUTPUT')
print('HIGH',cloud_class_array.clch.sel(ncells=slice(4642,4660)).squeeze().values)
print('MID' ,cloud_class_array.clcm.sel(ncells=slice(4642,4660)).squeeze().values)
print('LOW' ,cloud_class_array.clcl.sel(ncells=slice(4642,4660)).squeeze().values)

print('HIGH',cloud_class_number.clch.sel(ncells=slice(4642,4660)).squeeze().values)
print('MID' ,cloud_class_number.clcm.sel(ncells=slice(4642,4660)).squeeze().values)
print('LOW' ,cloud_class_number.clcl.sel(ncells=slice(4642,4660)).squeeze().values)
print('###################################################################')

#cloud_class_number

#print('I save the cloud class number')
#opath= '/pf/b/b380796/scratch/hackathon/george/'
#np.save(opath+'cloud_class_array_thres10p_2km_alltimesteps_v2',cloud_class_number.squeeze().values)